<a href="https://colab.research.google.com/github/jacquie0583/Amazon_Vine_Analysis/blob/main/Part_2__Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [67.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 h

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-09-27 03:56:24--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2021-09-27 03:56:24 (11.3 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

In [ ]:
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...| 2015-08-31|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...|          Sports|          5|    

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1WBPB8MDCCN8F|          5|            0|          0|   N|                N|
|R32M0YEWV77XG8|          5|            1|          1|   N|                Y|
| RR8V7WR27NXJ5|          1|            0|          0|   N|                Y|
|R1MHO5V9Z932AY|          5|            0|          0|   N|                Y|
|R16PD71086BD2V|          5|            0|          1|   N|                N|
|R1Z8IFGWTRWXT6|          3|            0|          0|   N|                Y|
|R3AUMSHAW73HWN|          4|            2|          3|   N|                N|
|R2KWDWFOHGX6FL|          5|            0|          0|   N|                Y|
|R3H9543FWBWFBU|          5|            1|          1|   N|                Y|
| RUANXOQ9W3OU5|          5|            0|          0|   N|     

In [ ]:
# Step 1: Create a new DF: condition => total_votes>=20
filtered_vine_df = vine_df.filter('total_votes>=20')
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|
|R18BTOE5UGN7UA|          5|           26|         28|   N|                Y|
|R1DXLGDSK2WMLD|          4|           22|         22|   N|                N|
| RZG2OHQ12YYZI|          5|           30|         31|   N|     

In [ ]:
# Step 2: Create a new DF: condition => helpful_votes/total_votes >= 50%
helpful_votes50_df = filtered_vine_df.withColumn('grater50', filtered_vine_df['helpful_votes'] / filtered_vine_df['total_votes']).filter('grater50>=0.5')
helpful_votes50_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|          grater50|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|               1.0|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|               1.0|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|               0.8|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|           0.96875|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|0.7384615384615385|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|0.9325842696629213|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|0.7727272727272727|
|R18BTOE5UGN7UA|          5|  

In [ ]:
print(helpful_votes50_df.count())

61948


In [ ]:
# Step 3: Create a new DF: condition => vine=='Y'
vine_Y_df = helpful_votes50_df.filter(helpful_votes50_df['vine'] == 'Y')
vine_Y_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|          grater50|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R32OMS1LSDRCOB|          4|           27|         31|   Y|                N|0.8709677419354839|
|R2IPKK4I0XRCFR|          4|           16|         21|   Y|                N|0.7619047619047619|
| RXYYJ8SGI9IX6|          5|           36|         41|   Y|                N|0.8780487804878049|
|R1SAUBFC7T6JHC|          5|           73|         75|   Y|                N|0.9733333333333334|
|R329HVJKKHWSG3|          5|           59|         59|   Y|                N|               1.0|
| RZL21K50M52KW|          5|           22|         22|   Y|                N|               1.0|
|R24RC5J2D92SBE|          5|           19|         20|   Y|                N|              0.95|
|R1803HITBY7Y08|          4|  

In [ ]:
# Step 4: Create a new DF: condition => vine=='N'
vine_N_df = helpful_votes50_df.filter(helpful_votes50_df['vine'] == 'N')
vine_N_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|          grater50|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R1B4753QMTD8ON|          5|           32|         32|   N|                N|               1.0|
|R28BVHOEL6GCHZ|          5|           21|         21|   N|                Y|               1.0|
|R27IB4BRKO71I0|          2|           16|         20|   N|                Y|               0.8|
| RJ86OW12Y4NRL|          5|          124|        128|   N|                Y|           0.96875|
|R3KRABOWCH6ZE4|          1|           48|         65|   N|                Y|0.7384615384615385|
|R1OAKARA30DIWX|          4|           83|         89|   N|                Y|0.9325842696629213|
|R36LSV4WQ4VB1K|          5|           17|         22|   N|                N|0.7727272727272727|
|R18BTOE5UGN7UA|          5|  

In [ ]:
from pyspark.sql.functions import count
#Total number of review for each dataframe from Step3 and Step4:
vine_Y_count = vine_Y_df.count()
print("Total Number of Reviews PAID and 'helpful': %f" % vine_Y_count)

vine_N_count = vine_N_df.count()
print("Total Number of Reviews NON-PAID and 'helpful': %f" % vine_N_count)

Total Number of Reviews PAID and 'helpful': 334.000000
Total Number of Reviews NON-PAID and 'helpful': 61614.000000


In [ ]:
# Number of 5-star Reviews for above Paid and Non-Paid dataframes
star5_vine_df = vine_Y_df.filter(vine_N_df["star_rating"]=="5")
print("Total Number of 5-star Reviews PAID and 'helpful': %f" % star5_vine_df.count())


star5_vine_N_df = vine_N_df.filter(vine_N_df["star_rating"]=="5")
print("Total Number of 5-star Reviews NON-PAID and 'helpful': %f" % star5_vine_N_df.count())

Total Number of 5-star Reviews PAID and 'helpful': 139.000000
Total Number of 5-star Reviews NON-PAID and 'helpful': 32665.000000


In [ ]:
# Step 5.1: vine_reviews_YES: 'total number of reviews' & 'number of 5-stars reviews' & '% of 5-stars reviews'
# Step 5.1.a: total number of reviews
vineY_reviews_TOTAL_df = vine_Y_df.agg({'review_id':'count'}).withColumnRenamed("count(review_id)", "Total_Vine_Reviews")
vineY_reviews_TOTAL_df.show()

+------------------+
|Total_Vine_Reviews|
+------------------+
|               334|
+------------------+



In [ ]:
# Step 5.1.b: 5-star revievs 
vineY_reviews_5STAR_df = vine_Y_df.filter('star_rating==5').agg({'star_rating':'count'}).withColumnRenamed("count(star_rating)", "5-stars_Vine_Reviews")
vineY_reviews_5STAR_df.show()

+--------------------+
|5-stars_Vine_Reviews|
+--------------------+
|                 139|
+--------------------+



In [ ]:
# Create data frame for % results
percentage_df_summ2 = spark.createDataFrame([
                                       (139, 32665)],
                                      ['5-star Total Vine Reviews', "5-star Total non-Vine Reviews"])
percentage_df_summ2.show()

+-------------------------+-----------------------------+
|5-star Total Vine Reviews|5-star Total non-Vine Reviews|
+-------------------------+-----------------------------+
|                      139|                        32665|
+-------------------------+-----------------------------+



In [ ]:
# Percentage Vine Reviews
vineY_reviews_percentage = vineY_reviews_5STAR_df.collect()[0]['5-stars_Vine_Reviews'] / vineY_reviews_TOTAL_df.collect()[0]['Total_Vine_Reviews'] * 100
round(vineY_reviews_percentage,2)

41.62

In [ ]:
# Step 5.2: vine_reviews_NO: 'total number of reviews' & 'number of 5-stars reviews' & '% of 5-stars reviews '
# Step 5.2.a: total number of reviews
vineN_reviews_TOTAL_df = vine_N_df.agg({'review_id':'count'}).withColumnRenamed("count(review_id)", "Total_non-Vine_Reviews")
vineN_reviews_TOTAL_df.show()

+----------------------+
|Total_non-Vine_Reviews|
+----------------------+
|                 61614|
+----------------------+



In [ ]:
# Step 5.2.a: number of 5-stars reviews
vineN_reviews_5STAR_df = vine_N_df.filter('star_rating==5').agg({'star_rating':'count'}).withColumnRenamed("count(star_rating)", "5-stars_non-Vine_Reviews")
vineN_reviews_5STAR_df.show()

+------------------------+
|5-stars_non-Vine_Reviews|
+------------------------+
|                   32665|
+------------------------+



In [ ]:
#  Percentage non-Vine Reviews
vineN_reviews_percentage = vineN_reviews_5STAR_df.collect()[0]['5-stars_non-Vine_Reviews'] / vineN_reviews_TOTAL_df.collect()[0]['Total_non-Vine_Reviews'] * 100
round(vineN_reviews_percentage,2)

53.02

In [ ]:
# Create data frame for % results
percentage_df_summ = spark.createDataFrame([
                                       (334, 61614)],
                                      ['Total Vine Reviews', "Total non-Vine Reviews"])
percentage_df_summ.show()

+------------------+----------------------+
|Total Vine Reviews|Total non-Vine Reviews|
+------------------+----------------------+
|               334|                 61614|
+------------------+----------------------+



In [ ]:
# Create data frame for % results
percentage_df = spark.createDataFrame([
                                       (41.62, 53.02)],
                                      ['Percentage Vine Reviews', "Percentage non-Vine Reviews"])
percentage_df.show()

+-----------------------+---------------------------+
|Percentage Vine Reviews|Percentage non-Vine Reviews|
+-----------------------+---------------------------+
|                  41.62|                      53.02|
+-----------------------+---------------------------+



In [ ]:
# Positivity Bias - further analysis
# Vine Reviews
all_stars_YES = vine_Y_df.groupby("star_rating").agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "Count of Reviews Vine")

In [ ]:
# Vine Reviews 
import pyspark.sql.functions as func
all_stars_YES_percentage = all_stars_YES.withColumn('% of total', func.round(all_stars_YES['Count of Reviews Vine']/ \
                                                (vineY_reviews_TOTAL_df.collect()[0]['Total_Vine_Reviews']/100), 2)) \
                                                .orderBy(all_stars_YES['star_rating'].asc()).show()

+-----------+---------------------+----------+
|star_rating|Count of Reviews Vine|% of total|
+-----------+---------------------+----------+
|          1|                    8|       2.4|
|          2|                   13|      3.89|
|          3|                   56|     16.77|
|          4|                  118|     35.33|
|          5|                  139|     41.62|
+-----------+---------------------+----------+



In [ ]:
# non-Vine Reviews
all_stars_NO = vine_N_df.groupby("star_rating").agg({"review_id":"count"}).withColumnRenamed("count(review_id)", "Count of Reviews non-Vine")

In [ ]:
# non-Vine Reviews (continued)
all_stars_NO_percentage = all_stars_NO.withColumn('% of total', func.round(all_stars_NO['Count of Reviews non-Vine']/ \
                                            (vineN_reviews_TOTAL_df.collect()[0]['Total_non-Vine_Reviews']/100), 2))  \
                                            .orderBy(all_stars_NO['star_rating'].asc()).show()

+-----------+-------------------------+----------+
|star_rating|Count of Reviews non-Vine|% of total|
+-----------+-------------------------+----------+
|          1|                     9192|     14.92|
|          2|                     3592|      5.83|
|          3|                     5477|      8.89|
|          4|                    10688|     17.35|
|          5|                    32665|     53.02|
+-----------+-------------------------+----------+



In [ ]:
from scipy.stats import chi2_contingency
import numpy as np
obs = np.array([[.42],[.53]])
stat2, p, dof,expected = chi2_contingency(obs)

In [ ]:
from scipy.stats import chi2
prob = .95 
critical = chi2.ppf(prob, dof)
if abs(stat2)>= critical: 
  print("significant")
else: 
  print('not significant')

not significant
